In [346]:
import pandas as pd
import numpy as np
import re, math
from string import punctuation

In [347]:
df = pd.read_excel("./data/Eni_Shell_data.xlsx")

In [348]:
df.shape

(1586, 46)

In [349]:
df.columns

Index(['Oil Spill ID', 'Company', 'JIV Number', 'Date Reported (Shell, Eni)',
       'Year', 'JIV Date (Shell)',
       'Facility equipment/ Incident site (Shell, Eni)',
       'Area/ Terrain (Shell, Eni)', 'Cause (Shell, Eni)',
       'Volume (Shell, Eni) barrels\n', 'Clean-up Status (Shell)',
       'Comments (Shell)', 'LGA (Eni)', 'JIV Asset Id', 'Included', 'JIV Url',
       'JIV URL hyperlinked', 'JIV Cause Verified', 'Date Incident',
       'Date start investigation', 'Description of Leak Point',
       'Incident caused by', 'Incident caused by (Clean)',
       'Comments cause (amnesty classification)',
       'Cause (amnesty classification)', 'JIV Location Verified',
       'Location.Type', 'Location Unit', 'Lat/Northing', 'Long/Easting',
       'Tranformation notes', 'Latitude (normalised)',
       'Longitude (normalised)', 'Area', 'AreaUnit', 'JIV Comment Type',
       'JIV Comment', 'photo_lookup_id', 'Photo Asset Id', 'Included.1',
       'Photo Url', 'Photo url hyperlinked'

### I. Nornalize column names

In [350]:
column_names = [
    "oil_spill_id",
    "company", 
    "jiv_number",
    "date_reported",
    "year",
    "date_jiv_shell",
    "facility_equipment",
    "terrain",
    "cause",
    "barrels",
    "cleanup_status_text",
    "comments_shell",
    "lga_eni",
    "jiv_asset_id",
    "in_decoders",
    "jiv_url",
    "jiv_url_hyperlinked",
    "cause_jiv_verified",
    "date_incident",
    "date_investigation_start",
    "leak_point_text",
    "cause_incident_caused_by_dirty",
    "cause_incident_caused_by",
    "cause_amnesty_comment",
    "cause_amnesty",
    "location_jiv_verified",
    "location_type",
    "location_unit",
    "lat_northing",
    "long_eastling",
    "location_transformation_notes",
    "lat",
    "long",
    "area_decoders",
    "area_unit",
    "comment_type_jiv",
    "comment_jiv",
    "photo_lookup_id",
    "photo_asset_id",
    "in_decoders2",
    "photo_url",
    "photo_url_hyperlinked",
    "damage_photo",
    "damage_photo_followup",
    "comment_jiv_duplicate",
    "comment_jiv_text"
]

In [351]:
pd.DataFrame({"Original Column Names": df.columns, "Alias": column_names}).to_csv("columns.csv")

In [352]:
df.columns.shape, len(column_names)

((46,), 46)

In [353]:
df.columns = column_names

### II. Explore data

#### 1. Do "Included" and "Included.1" hold the same data?

In [354]:
df[["in_decoders", "in_decoders2"]].head()

,in_decoders,in_decoders2
0,Y,Y
1,Y,Y
2,N,Y
3,N,Y
4,Y,Y


In [355]:
df.in_decoders2.value_counts()

Y                                       1558
N (asset error on shell/eni wensite)      13
Missing photo                              8
N                                          5
N, photo missing                           2
Name: in_decoders2, dtype: int64

#### 2. explore Facility Equiptment

- '' inches
- pipeline name
- "at" location

In [356]:
df.facility_equipment.head(5)

0                            24'' Ogoda/Brass Pipeline
1    12'' Imo River 1 and 2 - Ogale Pipeline at komkom
2              Adibawa Well 8 S/L Wellhead at Edagberi
3           24'' Bomu - Bonny Pipeline at Okolo Launch
4       10'' Diebu Creek-Nun River Pipeline at Oporoma
Name: facility_equipment, dtype: object

In [357]:
# regexs
inch_single_quote_regex = re.compile(r"(\d+)''")
inch_double_quote_regex = re.compile(r'(\d+)"')
location_regex = re.compile(r"at\s(.*)")

inches = [np.nan] * df.shape[0]
facility_type_name = [np.nan] * df.shape[0]
facility_location = [np.nan] * df.shape[0]

no_inch_cnt = 0
missing_loc_count = 0
for i in range(df.shape[0]):
    
    facility_info = df.facility_equipment.iloc[i].lower()
    # a. exctract inches
    try:
        inches[i] = int(re.search(inch_single_quote_regex, facility_info).group(1))
    except:
        try:
            inches[i] = int(re.search(inch_double_quote_regex, facility_info).group(1))
        except:
            no_inch_cnt += 1
            
    # b. extract facility type
    type_found = False
    otherline_seen = "line" in facility_info
    flowlines = set(["flowline", "fl"])
    flowline_seen = flowlines.intersection(set(facility_info.split(" "))) or "flow line" in facility_info
    pipeline_seen = "pipeline" in facility_info
    well_seen = "well" in facility_info
    wellhead_seen = "wellhead" in facility_info or "well head" in facility_info
    manifold_seen = "manifold" in facility_info
    trunklines = set(["trunkline", "tl"])
    trunkline_seen = trunklines.intersection(set(facility_info.split(" "))) or "trunk line" in facility_info
    deliverylines = set(["deliveryline", "dl"])
    deliveryline_seen = deliverylines.intersection(set(facility_info.split(" "))) or "delivery line" in facility_info
    bulklines = set(["bulkline", "bl"])
    bulkline_seen = bulklines.intersection(set(facility_info.split(" "))) or "bulk line" in facility_info
    flowstation_seen = "flowstation" in facility_info or "flow station" in facility_info
 
    if otherline_seen:
        facility_type_name[i] = "other line"
        type_found = True
    
    if pipeline_seen and not well_seen:
        facility_type_name[i] = "pipeline"
        type_found = True
       
    if flowline_seen and "well" in facility_info:
        facility_type_name[i] = "flowline, well"
        type_found = True
        
    if flowline_seen and not well_seen:
        facility_type_name[i] = "flowline"
        type_found = True
        
    if well_seen and not wellhead_seen and not flowline_seen and not pipeline_seen:
        facility_type_name[i] = "well"
        type_found = True
        
    if wellhead_seen:
        facility_type_name[i] = "wellhead"
        type_found = True
        
    if manifold_seen:
        facility_type_name[i] = "manifold"
        type_found = True

    if trunkline_seen:
        facility_type_name[i] = "trunkline"
        type_found = True

    if deliveryline_seen:
        facility_type_name[i] = "deliveryline"
        type_found = True
        
    if bulkline_seen:
        facility_type_name[i] = "bulkline"
        type_found = True
    
    if flowstation_seen:
        facility_type_name[i] = "flowstation"
        type_found = True
    
    if not type_found:
        facility_type_name[i] = "other"
        
    # c. extract location
    try:
        facility_location[i] = str(re.search(location_regex, facility_info).group(1)).strip(punctuation)
    except:
        missing_loc_count +=1

df["inches"] = pd.Series(inches)
df["facility_type"] = pd.Series(facility_type_name)
df["facility_location"] = pd.Series(facility_location)

In [358]:
df.facility_type.value_counts()

pipeline          933
flowline, well    179
flowline          149
wellhead           64
well               52
other              46
trunkline          45
deliveryline       45
manifold           31
flowstation        21
other line         14
bulkline            7
Name: facility_type, dtype: int64

In [359]:
df.facility_location.isnull().sum()

761

In [360]:
df[["facility_location", "lga_eni"]].head()

,facility_location,lga_eni
0,NaN,Abua/Odual
1,komkom,NaN
2,edagberi,NaN
3,okolo launch,NaN
4,oporoma,NaN


#### * check if when Shell location is given, Eni is missing and vice versa

In [361]:
# shell location -> facility_location
# eni location -> lga_eni
shell_loc = [0] * df.shape[0]
eni_loc = [0] * df.shape[0]

for i in range(df.shape[0]):
    shell_cur_loc = df.facility_location.iloc[i]
    eni_curr_loc = df.lga_eni.iloc[i]
    if type(shell_cur_loc) != float:
        shell_loc[i] = 1
    if type(eni_curr_loc) != float:
        eni_loc[i] = 1
        
shell_loc_series = pd.Series(shell_loc)
eni_loc_series = pd.Series(eni_loc)
loc_series = shell_loc_series.add(eni_loc_series)
loc_series.value_counts()

1    1522
0      59
2       5
dtype: int64

In [362]:
df["location_joint"] = pd.Series(loc_series)

In [363]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 2]

,oil_spill_id,company,facility_location,lga_eni
752,385,NAOC,the mini manifold,Southern Ijaw
1156,771,NAOC,olugbobiri,Southern Ijaw
1363,703,NAOC,obama 3 deep slot,Southern Ijaw
1372,1494,NAOC,oshie flow station,Oshie
1373,1202,NAOC,obiafu 14 well location,Ogba/Ndoni/Egbema


In [364]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 1].head()

,oil_spill_id,company,facility_location,lga_eni
0,1272,NAOC,NaN,Abua/Odual
1,221,SPDC,komkom,NaN
2,640,SPDC,edagberi,NaN
3,1109,SPDC,okolo launch,NaN
4,486,SPDC,oporoma,NaN


#### * clean the facility_locations for ENI

In [365]:
df.facility_location[df.location_joint == 2] = np.nan

/home/katia/.virtualenvs/securaxis/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [366]:
df[["oil_spill_id", "company","facility_location", "lga_eni"]][df.location_joint == 2]

,oil_spill_id,company,facility_location,lga_eni
752,385,NAOC,NaN,Southern Ijaw
1156,771,NAOC,NaN,Southern Ijaw
1363,703,NAOC,NaN,Southern Ijaw
1372,1494,NAOC,NaN,Oshie
1373,1202,NAOC,NaN,Ogba/Ndoni/Egbema


In [367]:
del df["location_joint"]

#### * aggregate Shell and Eni locations in the same column

In [368]:
locations = [np.nan] * df.shape[0]
for i in range(df.shape[0]):
    shell_loc = df.facility_location.iloc[i]
    eni_loc = df.lga_eni.iloc[i]
    if type(eni_loc) == float:
        locations[i] = shell_loc
    else:
        locations[i] = eni_loc
        
df['location'] = pd.Series(locations).values

In [369]:
del df["lga_eni"]
del df["facility_location"]

#### * extract the pipeline name

In [370]:
# handles from and to loc in cases such as: "nun-river kolo creek" or "biedu - nun-river"
def adjust_splits(splitted_list):
    from_loc = ""
    to_loc = ""
    if splitted_list[1] == "river" or splitted_list[1] == "creek":
        from_loc = splitted_list[0] + " " + splitted_list[1]
        to_loc = " ".join(splitted_list[2:])
    else:
        from_loc = splitted_list[0]
        to_loc = " ".join(splitted_list[1:])
    return from_loc, to_loc

In [371]:
df["facility_equipment_lower"] = df.facility_equipment.str.lower()
facility_names = [np.nan] * df.shape[0]
facility_start = [np.nan] * df.shape[0]
facility_end = [np.nan] * df.shape[0]

missing_fac_name_cnt = 0
for i in range(df.shape[0]):
    facility = df.facility_equipment_lower.iloc[i]
    current_facility_type = df.facility_type.iloc[i]
    # string replacements: "kolocreek" for "kolo creek", and "imo river ii" for "imo river 2"
    facility = facility.replace("kolocreek", "kolo creek")
    facility = facility.replace("imo river ii", "imo river 2")
    facility = facility.replace("imo river i", "imo river 1")

        
    if current_facility_type != "other":
        current_inches = df.inches.iloc[i]
        curr_type__abbr = current_facility_type[:5]
        # find the start index of the substring
        if math.isnan(current_inches):
            # start extracting from index 0
            start_index = 0
        else:
            # start extracting from the index of the word after the first space
            start_index = facility.find(" ") + 1
        # find the end index of the substring
        end_index = facility.find(curr_type__abbr)
        if end_index != -1:
            current_facility_name = facility[start_index:end_index-1]
            current_facility_name = current_facility_name.replace("  ", " ") # replace double space with single
            facility_names[i] = current_facility_name.strip()
            
            if current_facility_type not in ["well", "wellhead", "flowline", "flowline, well"]:

                dash_split = current_facility_name.split("-")
                to_split = current_facility_name.split(" to ")
                baskslash_split = current_facility_name.split("/")

                from_loc = ""
                to_loc = ""
                if len(dash_split) > 1:
    #                 if current_facility_name == "nun-river - kolo creek":
    #                     print("yes", dash_split, list(map(str.strip, dash_split)))
                    from_loc, to_loc = adjust_splits(list(map(str.strip, dash_split)))

                if len(baskslash_split) > 1:
                    from_loc, to_loc = adjust_splits(list(map(str.strip, baskslash_split)))

                if len(to_split) > 1:
                    from_loc, to_loc = adjust_splits(list(map(str.strip, to_split)))


                facility_start[i] = from_loc
                facility_end[i] = to_loc
            
        else:
            missing_fac_name_cnt += 1
            
print("There are", missing_fac_name_cnt, "missing facility names.")
df["facility_name"] = pd.Series(facility_names).values
df["facility_start"] = pd.Series(facility_start).values
df["facility_end"] = pd.Series(facility_end).values

There are 61 missing facility names.


In [372]:
del df["facility_equipment_lower"]
df[["facility_equipment", "inches", "facility_name", "facility_start", "facility_end", "facility_type", "location"]].to_csv("./data/EniShell_transformed.csv")

### 3. Verify company

In [373]:
df.company.value_counts()

SPDC    865
NAOC    721
Name: company, dtype: int64

### 4. Inspect cause

#### * map Shell/Eni causes to 3 large categories

In [427]:
df.groupby( [ "company", "cause"] ).count()

oil_spill_id  jiv_number  date_reported  \
company cause                                                                  
NAOC    Corrosion                              16          16             16   
        Drilled hole                           24          24             24   
        Equipment failure                      61          61             61   
        Hacksaw cut                           106         106            106   
        Hacksaw cut & explosive                 4           4              4   
        Hacksaw cut & fire                      1           1              1   
        Induced corrosion                       4           4              4   
        Oil theft                             409         409            409   
        Operational error                       2           2              2   
        Operational error/Oil theft             1           1              1   
        Other                                   2           2              2   
        Road Traffic Accident                   1           1              1   
        Sabotage                                2           2              2   
        Structure failure                       2           2              2   
        Unknown                                 1           1              1   
        Use of explosive                       41          41             41   
        Vandalization                          44          44             44   
SPDC    Mystery Spill                           3           3              3   
        Operational                           164         160            164   
        Other                                   3           3              3   
        Sabotage                              468         452            468   
        Sabotage/ Theft                       227         218            227   

                                     year  date_jiv_shell  facility_equipment  \
company cause                                                                   
NAOC    Corrosion                      16               0                  16   
        Drilled hole                   24               0                  24   
        Equipment failure              61               0                  61   
        Hacksaw cut                   106               0                 106   
        Hacksaw cut & explosive         4               0                   4   
        Hacksaw cut & fire              1               0                   1   
        Induced corrosion               4               0                   4   
        Oil theft                     409               0                 409   
        Operational error               2               0                   2   
        Operational error/Oil theft     1               0                   1   
        Other                           2               0                   2   
        Road Traffic Accident           1               0                   1   
        Sabotage                        2               0                   2   
        Structure failure               2               0                   2   
        Unknown                         1               0                   1   
        Use of explosive               41               0                  41   
        Vandalization                  44               0                  44   
SPDC    Mystery Spill                   3               3                   3   
        Operational                   164             164                 164   
        Other                           3               3                   3   
        Sabotage                      468             468                 468   
        Sabotage/ Theft               227             227                 227   

                                     terrain  barrels  cleanup_status_text  \
company cause                                                                
NAOC    Corrosi

In [426]:
#### * fit Shell/Eni cause to 3 categories
sabotage_theft = ["Sabotage", "Sabotage/ Theft", "Hacksaw cut", "Vandalization", "Use of explosive", "Hacksaw cut & explosive", "Hacksaw cut & fire", "Oil theft", "Drilled hole"]
company_fault = ["Operational", "Equipment failure", "Corrosion", "Induced corrosion", "Operational error/Oil theft", "Structure failure", "Operational error" ]
other = ["Other", "Mystery Spill", "Road Traffic Accident", "Unknown"]

mapped_cuased = [""] * df.shape[0]
for i in range(df.shape[0]):
    reported_cause = df.cause.iloc[i]
    if reported_cause in sabotage_theft:
        mapped_cuased[i] = "sabotage/theft"
    if reported_cause in company_fault:
        mapped_cuased[i] = "company's fault"
    if reported_cause in other:
        mapped_cuased[i] = "other"
        
df["cause_mapped"] = pd.Series(mapped_cuased).values
df[:].groupby(["cause_mapped", "company"]).count()

oil_spill_id  jiv_number  date_reported  year  \
cause_mapped    company                                                  
company's fault NAOC               86          86             86    86   
                SPDC              164         160            164   164   
other           NAOC                4           4              4     4   
                SPDC                6           6              6     6   
sabotage/theft  NAOC              631         631            631   631   
                SPDC              695         670            695   695   

                         date_jiv_shell  facility_equipment  terrain  cause  \
cause_mapped    company                                                       
company's fault NAOC                  0                  86       86     86   
                SPDC                164                 164      164    164   
other           NAOC                  0                   4        4      4   
                SPDC                  6                   6        6      6   
sabotage/theft  NAOC                  0                 631      631    631   
                SPDC                695                 695      695    695   

                         barrels  cleanup_status_text      ...       \
cause_mapped    company                                    ...        
company's fault NAOC          86                    0      ...        
                SPDC         164                  164      ...        
other           NAOC           4                    0      ...        
                SPDC           6                    6      ...        
sabotage/theft  NAOC         631                    0      ...        
                SPDC         695                  695      ...        

                         damage_photo  damage_photo_followup  \
cause_mapped    company                                        
company's fault NAOC               86                     77   
                SPDC              164                    146   
other           NAOC                4                      3   
                SPDC                6                      5   
sabotage/theft  NAOC              631                    107   
                SPDC              695                    144   

                         comment_jiv_duplicate  comment_jiv_text  inches  \
cause_mapped    company                                                    
company's fault NAOC                        68                68      39   
                SPDC                        83                83      71   
other           NAOC                         3                 3       3   
                SPDC                         5                 5       3   
sabotage/theft  NAOC                       384               384     508   
                SPDC                       161               161     556   

                         facility_type  location  facility_name  \
cause_mapped    company                                           
company's fault NAOC                86        84             73   
                SPDC               164       145            141   
other           NAOC                 4         4              2   
                SPDC                 6         6              5   
sabotage/theft  NAOC               631       619            616   
                SPDC               695       669            642   

                         facility_start  facility_end  
cause_mapped    company                                
company's fault NAOC                 30            30  
                SPDC                 62            62  
other           NAOC                  2             2  
                SPDC                  2             2  
sabotage/theft  NAOC                477           477  
                SPDC                495           495  

[6 rows x 50 columns]

In [429]:
df[:].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  \
company cause_mapped    cause_amnesty                                    
NAOC    company's fault Operational                                 76   
                        Operational; Third party (theft)             1   
                        Third party (theft)                          2   
                        Third party (undetermined)                   1   
                        Undetermined                                 6   
        other           Operational                                  1   
                        Third party (accident)                       1   
                        Third party (undetermined)                   1   
                        Undetermined                                 1   
        sabotage/theft  Operational                                 66   
                        Third party (theft)                        393   
                        Third party (undetermined)                 132   
                        Undetermined                                40   
SPDC    company's fault Operational                                152   
                        Third party (accident)                       1   
                        Third party (undetermined)                   4   
                        Undetermined                                 7   
        other           Operational                                  1   
                        Third party (accident)                       1   
                        Undetermined                                 4   
        sabotage/theft  Operational                                  7   
                        Third party (theft)                        377   
                        Third party (undetermined)                 174   
                        Undetermined                               137   

                                                          jiv_number  \
company cause_mapped    cause_amnesty                                  
NAOC    company's fault Operational                               76   
                        Operational; Third party (theft)           1   
                        Third party (theft)                        2   
                        Third party (undetermined)                 1   
                        Undetermined                               6   
        other           Operational                                1   
                        Third party (accident)                     1   
                        Third party (undetermined)                 1   
                        Undetermined                               1   
        sabotage/theft  Operational                               66   
                        Third party (theft)                      393   
                        Third party (undetermined)               132   
                        Undetermined                              40   
SPDC    company's fault Operational                              152   
                        Third party (accident)                     1   
                        Third party (undetermined)                 4   
                        Undetermined                               3   
        other           Operational                                1   
                        Third party (accident)                     1   
                        Undetermined                               4   
        sabotage/theft  Operational                                7   
                        Third party (theft)                      372   
                        Third party (undetermined)               167   
                        Undetermined                             124   

                                                          date_reported  year  \
company cause_mapped    cause_amnesty                                           
NAOC    company's fault Operational                                  76    76   
                 

#### * compare Decoder's cause to 3 large categories

In [433]:
df.cause_amnesty.value_counts()

Third party (theft)                 772
Third party (undetermined)          312
Operational                         303
Undetermined                        195
Third party (accident)                3
Operational; Third party (theft)      1
Name: cause_amnesty, dtype: int64

In [437]:
df[df.cause_amnesty == "Operational"].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  jiv_number  \
company cause_mapped    cause_amnesty                             
NAOC    company's fault Operational              76          76   
        other           Operational               1           1   
        sabotage/theft  Operational              66          66   
SPDC    company's fault Operational             152         152   
        other           Operational               1           1   
        sabotage/theft  Operational               7           7   

                                       date_reported  year  date_jiv_shell  \
company cause_mapped    cause_amnesty                                        
NAOC    company's fault Operational               76    76               0   
        other           Operational                1     1               0   
        sabotage/theft  Operational               66    66               0   
SPDC    company's fault Operational              152   152             152   
        other           Operational                1     1               1   
        sabotage/theft  Operational                7     7               7   

                                       facility_equipment  terrain  cause  \
company cause_mapped    cause_amnesty                                       
NAOC    company's fault Operational                    76       76     76   
        other           Operational                     1        1      1   
        sabotage/theft  Operational                    66       66     66   
SPDC    company's fault Operational                   152      152    152   
        other           Operational                     1        1      1   
        sabotage/theft  Operational                     7        7      7   

                                       barrels  cleanup_status_text  \
company cause_mapped    cause_amnesty                                 
NAOC    company's fault Operational         76                    0   
        other           Operational          1                    0   
        sabotage/theft  Operational         66                    0   
SPDC    company's fault Operational        152                  152   
        other           Operational          1                    1   
        sabotage/theft  Operational          7                    7   

                                           ...       damage_photo  \
company cause_mapped    cause_amnesty      ...                      
NAOC    company's fault Operational        ...                 76   
        other           Operational        ...                  1   
        sabotage/theft  Operational        ...                 66   
SPDC    company's fault Operational        ...                152   
        other           Operational        ...                  1   
        sabotage/theft  Operational        ...                  7   

                                       damage_photo_followup  \
company cause_mapped    cause_amnesty                          
NAOC    company's fault Operational                       71   
        other           Operational                        1   
        sabotage/theft  Operational                       65   
SPDC    company's fault Operational                      138   
        other           Operational                        0   
        sabotage/theft  Operational                        6   

                                       comment_jiv_duplicate  \
company cause_mapped    cause_amnesty                          
NAOC    company's fault Operational                       61   
        other           Operational                        1   
        sabotage/theft  Operational                       49   
SPDC    company's fault Operational                       76   
        other           Operational                        1   
        sabotage/theft  Operational                        3   

                                       comment_jiv_text  inches  \
company cause_mapped    cause_amnesty        

In [434]:
df[df.cause_amnesty == "Undetermined"].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  jiv_number  \
company cause_mapped    cause_amnesty                             
NAOC    company's fault Undetermined              6           6   
        other           Undetermined              1           1   
        sabotage/theft  Undetermined             40          40   
SPDC    company's fault Undetermined              7           3   
        other           Undetermined              4           4   
        sabotage/theft  Undetermined            137         124   

                                       date_reported  year  date_jiv_shell  \
company cause_mapped    cause_amnesty                                        
NAOC    company's fault Undetermined               6     6               0   
        other           Undetermined               1     1               0   
        sabotage/theft  Undetermined              40    40               0   
SPDC    company's fault Undetermined               7     7               7   
        other           Undetermined               4     4               4   
        sabotage/theft  Undetermined             137   137             137   

                                       facility_equipment  terrain  cause  \
company cause_mapped    cause_amnesty                                       
NAOC    company's fault Undetermined                    6        6      6   
        other           Undetermined                    1        1      1   
        sabotage/theft  Undetermined                   40       40     40   
SPDC    company's fault Undetermined                    7        7      7   
        other           Undetermined                    4        4      4   
        sabotage/theft  Undetermined                  137      137    137   

                                       barrels  cleanup_status_text  \
company cause_mapped    cause_amnesty                                 
NAOC    company's fault Undetermined         6                    0   
        other           Undetermined         1                    0   
        sabotage/theft  Undetermined        40                    0   
SPDC    company's fault Undetermined         7                    7   
        other           Undetermined         4                    4   
        sabotage/theft  Undetermined       137                  137   

                                           ...       damage_photo  \
company cause_mapped    cause_amnesty      ...                      
NAOC    company's fault Undetermined       ...                  6   
        other           Undetermined       ...                  1   
        sabotage/theft  Undetermined       ...                 40   
SPDC    company's fault Undetermined       ...                  7   
        other           Undetermined       ...                  4   
        sabotage/theft  Undetermined       ...                137   

                                       damage_photo_followup  \
company cause_mapped    cause_amnesty                          
NAOC    company's fault Undetermined                       6   
        other           Undetermined                       1   
        sabotage/theft  Undetermined                      35   
SPDC    company's fault Undetermined                       7   
        other           Undetermined                       4   
        sabotage/theft  Undetermined                     127   

                                       comment_jiv_duplicate  \
company cause_mapped    cause_amnesty                          
NAOC    company's fault Undetermined                       4   
        other           Undetermined                       1   
        sabotage/theft  Undetermined                      19   
SPDC    company's fault Undetermined                       3   
        other           Undetermined                       4   
        sabotage/theft  Undetermined                      45   

                                       comment_jiv_text  inches  \
company cause_mapped    cause_amnesty        

In [436]:
df[df.cause_amnesty == "Third party (undetermined)"].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  jiv_number  \
company cause_mapped    cause_amnesty                                          
NAOC    company's fault Third party (undetermined)             1           1   
        other           Third party (undetermined)             1           1   
        sabotage/theft  Third party (undetermined)           132         132   
SPDC    company's fault Third party (undetermined)             4           4   
        sabotage/theft  Third party (undetermined)           174         167   

                                                    date_reported  year  \
company cause_mapped    cause_amnesty                                     
NAOC    company's fault Third party (undetermined)              1     1   
        other           Third party (undetermined)              1     1   
        sabotage/theft  Third party (undetermined)            132   132   
SPDC    company's fault Third party (undetermined)              4     4   
        sabotage/theft  Third party (undetermined)            174   174   

                                                    date_jiv_shell  \
company cause_mapped    cause_amnesty                                
NAOC    company's fault Third party (undetermined)               0   
        other           Third party (undetermined)               0   
        sabotage/theft  Third party (undetermined)               0   
SPDC    company's fault Third party (undetermined)               4   
        sabotage/theft  Third party (undetermined)             174   

                                                    facility_equipment  \
company cause_mapped    cause_amnesty                                    
NAOC    company's fault Third party (undetermined)                   1   
        other           Third party (undetermined)                   1   
        sabotage/theft  Third party (undetermined)                 132   
SPDC    company's fault Third party (undetermined)                   4   
        sabotage/theft  Third party (undetermined)                 174   

                                                    terrain  cause  barrels  \
company cause_mapped    cause_amnesty                                         
NAOC    company's fault Third party (undetermined)        1      1        1   
        other           Third party (undetermined)        1      1        1   
        sabotage/theft  Third party (undetermined)      132    132      132   
SPDC    company's fault Third party (undetermined)        4      4        4   
        sabotage/theft  Third party (undetermined)      174    174      174   

                                                    cleanup_status_text  \
company cause_mapped    cause_amnesty                                     
NAOC    company's fault Third party (undetermined)                    0   
        other           Third party (undetermined)                    0   
        sabotage/theft  Third party (undetermined)                    0   
SPDC    company's fault Third party (undetermined)                    4   
        sabotage/theft  Third party (undetermined)                  174   

                                                        ...       \
company cause_mapped    cause_amnesty                   ...        
NAOC    company's fault Third party (undetermined)      ...        
        other           Third party (undetermined)      ...        
        sabotage/theft  Third party (undetermined)      ...        
SPDC    company's fault Third party (undetermined)      ...        
        sabotage/theft  Third party (undetermined)      ...        

                                                    damage_photo  \
company cause_mapped    cause_amnesty                              
NAOC    company's fault Third party (undetermined)             1   
        other           Third party (undetermined)             1   
        sabotage/theft  Third party (undetermined)           132   
SPDC    company's fault Third party (undetermined)        

In [435]:
df[df.cause_amnesty == "Third party (theft)"].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  jiv_number  \
company cause_mapped    cause_amnesty                                   
NAOC    company's fault Third party (theft)             2           2   
        sabotage/theft  Third party (theft)           393         393   
SPDC    sabotage/theft  Third party (theft)           377         372   

                                             date_reported  year  \
company cause_mapped    cause_amnesty                              
NAOC    company's fault Third party (theft)              2     2   
        sabotage/theft  Third party (theft)            393   393   
SPDC    sabotage/theft  Third party (theft)            377   377   

                                             date_jiv_shell  \
company cause_mapped    cause_amnesty                         
NAOC    company's fault Third party (theft)               0   
        sabotage/theft  Third party (theft)               0   
SPDC    sabotage/theft  Third party (theft)             377   

                                             facility_equipment  terrain  \
company cause_mapped    cause_amnesty                                      
NAOC    company's fault Third party (theft)                   2        2   
        sabotage/theft  Third party (theft)                 393      393   
SPDC    sabotage/theft  Third party (theft)                 377      377   

                                             cause  barrels  \
company cause_mapped    cause_amnesty                         
NAOC    company's fault Third party (theft)      2        2   
        sabotage/theft  Third party (theft)    393      393   
SPDC    sabotage/theft  Third party (theft)    377      377   

                                             cleanup_status_text  \
company cause_mapped    cause_amnesty                              
NAOC    company's fault Third party (theft)                    0   
        sabotage/theft  Third party (theft)                    0   
SPDC    sabotage/theft  Third party (theft)                  377   

                                                 ...       damage_photo  \
company cause_mapped    cause_amnesty            ...                      
NAOC    company's fault Third party (theft)      ...                  2   
        sabotage/theft  Third party (theft)      ...                393   
SPDC    sabotage/theft  Third party (theft)      ...                377   

                                             damage_photo_followup  \
company cause_mapped    cause_amnesty                                
NAOC    company's fault Third party (theft)                      0   
        sabotage/theft  Third party (theft)                      4   
SPDC    sabotage/theft  Third party (theft)                      7   

                                             comment_jiv_duplicate  \
company cause_mapped    cause_amnesty                                
NAOC    company's fault Third party (theft)                      2   
        sabotage/theft  Third party (theft)                    262   
SPDC    sabotage/theft  Third party (theft)                     73   

                                             comment_jiv_text  inches  \
company cause_mapped    cause_amnesty                                   
NAOC    company's fault Third party (theft)                 2       2   
        sabotage/theft  Third party (theft)               262     391   
SPDC    sabotage/theft  Third party (theft)                73     355   

                                             facility_type  location  \
company cause_mapped    cause_amnesty                                  
NAOC    company's fault Third party (theft)              2         2   
        sabotage/theft  Third party (theft)            393       387   
SPDC    sabotage/theft  Third party (theft)            377       364   

                                             facility_name  facility_start  \
company cause_mapped    cause_amnesty                                        
NAOC    company's fault

#### * compare "Equipment failure" to cause reported by Shell/Eni

In [438]:
df.cause_incident_caused_by.value_counts().head(10)

Third party interference                  405
 Third party interference; Crude theft    217
Third party interference                  213
 Third party interference                 210
 Third party interference; Sabotage       171
 Operational                              134
Equipment failure                          54
Missing JIV                                21
Corrosion                                  14
 Third party interference; crude theft     12
Name: cause_incident_caused_by, dtype: int64

In [439]:
df[df.cause_incident_caused_by == "Equipment failure"].groupby(["company", "cause_mapped", "cause_amnesty"]).count()

oil_spill_id  jiv_number  \
company cause_mapped    cause_amnesty                                   
NAOC    company's fault Operational                    51          51   
                        Third party (theft)             2           2   
        sabotage/theft  Operational                     1           1   

                                             date_reported  year  \
company cause_mapped    cause_amnesty                              
NAOC    company's fault Operational                     51    51   
                        Third party (theft)              2     2   
        sabotage/theft  Operational                      1     1   

                                             date_jiv_shell  \
company cause_mapped    cause_amnesty                         
NAOC    company's fault Operational                       0   
                        Third party (theft)               0   
        sabotage/theft  Operational                       0   

                                             facility_equipment  terrain  \
company cause_mapped    cause_amnesty                                      
NAOC    company's fault Operational                          51       51   
                        Third party (theft)                   2        2   
        sabotage/theft  Operational                           1        1   

                                             cause  barrels  \
company cause_mapped    cause_amnesty                         
NAOC    company's fault Operational             51       51   
                        Third party (theft)      2        2   
        sabotage/theft  Operational              1        1   

                                             cleanup_status_text  \
company cause_mapped    cause_amnesty                              
NAOC    company's fault Operational                            0   
                        Third party (theft)                    0   
        sabotage/theft  Operational                            0   

                                                 ...       damage_photo  \
company cause_mapped    cause_amnesty            ...                      
NAOC    company's fault Operational              ...                 51   
                        Third party (theft)      ...                  2   
        sabotage/theft  Operational              ...                  1   

                                             damage_photo_followup  \
company cause_mapped    cause_amnesty                                
NAOC    company's fault Operational                             48   
                        Third party (theft)                      0   
        sabotage/theft  Operational                              1   

                                             comment_jiv_duplicate  \
company cause_mapped    cause_amnesty                                
NAOC    company's fault Operational                             46   
                        Third party (theft)                      2   
        sabotage/theft  Operational                              1   

                                             comment_jiv_text  inches  \
company cause_mapped    cause_amnesty                                   
NAOC    company's fault Operational                        46      20   
                        Third party (theft)                 2       2   
        sabotage/theft  Operational                         1       0   

                                             facility_type  location  \
company cause_mapped    cause_amnesty                                  
NAOC    company's fault Operational                     51        50   
                        Third party (theft)              2         2   
        sabotage/theft  Operational                      1         1   

                                             facility_name  facility_start  \
company cause_mapped    cause_amnesty                                        
NAOC    company's fault

#### * process "leak Point"'s text

In [444]:
leak_points = set()
for i in range(df.shape[0]):
    lp = df.leak_point_text.iloc[i]
    if type(lp) != float:
        for point in lp.split(";"):
            leak_points.add(point.strip())

leak_points

{'',
 'Accidental 3rd party equipment impact',
 'Acid',
 'Blasting',
 'Bulging outward',
 'Complete Rupture',
 'Corrosion',
 'Corrupt JIV',
 'Crude oil theft/ illegal bunkering',
 'Drilled hole',
 'Drilling',
 'Explosive tear',
 'Failed clamp',
 'Failed weld on illegal hot tap valve',
 'Hack saw cut',
 'Inward dent',
 'Missing JIV',
 'Missing pipeline/ flowline',
 'Missing pipeline/flowline',
 'Other',
 'Others',
 'Outward dent',
 'Photo',
 'Saver pit over flow',
 'Sawing',
 'Tear',
 'Third party tampering with clamp',
 'Third party tampering with flange',
 'Third party tampering with valve',
 'Valve failure',
 'Well head tampering'}

In [445]:
df[["leak_point_text", "photo_url"]].to_csv("./data/leak_points.csv")